# SQL Window Functions: A Comprehensive Guide

Window functions are powerful SQL features that allow you to perform calculations across a set of table rows that are somehow related to the current row. Unlike aggregate functions, window functions don't cause rows to become grouped into a single output row — the rows retain their separate identities.

## What are Window Functions?

Window functions operate on a set of rows and return a single value for each row from the underlying query. The set of rows is called a "window" and is defined by the `OVER` clause.

### Basic Syntax
```sql
function_name([arguments]) OVER (
    [PARTITION BY column1, column2, ...]
    [ORDER BY column1 [ASC|DESC], column2 [ASC|DESC], ...]
    [ROWS|RANGE frame_specification]
)
```

## Setup: Import Required Libraries

First, let's import the necessary Python libraries to make this notebook interactive and runnable:

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Helper function to execute SQL and display results
def run_sql(query, description=None):
    if description:
        print(f"\n{description}")
        print("=" * len(description))
    
    try:
        result = pd.read_sql_query(query, conn)
        print(result.to_string(index=False))
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None

print("✅ Libraries imported and database connection established!")

✅ Libraries imported and database connection established!


## Sample Data Setup

Let's create some sample data to work with:

In [2]:
# Create the sales table
create_table_query = """
CREATE TABLE sales (
    employee_id INTEGER,
    employee_name TEXT,
    department TEXT,
    sale_date DATE,
    sale_amount REAL
);
"""

conn.execute(create_table_query)

# Insert sample data
insert_data_query = """
INSERT INTO sales VALUES
(1, 'Alice Johnson', 'Sales', '2024-01-15', 1500.00),
(2, 'Bob Smith', 'Sales', '2024-01-16', 2200.00),
(3, 'Carol Davis', 'Marketing', '2024-01-17', 1800.00),
(1, 'Alice Johnson', 'Sales', '2024-02-10', 1750.00),
(4, 'David Wilson', 'Marketing', '2024-02-12', 2100.00),
(2, 'Bob Smith', 'Sales', '2024-02-15', 1900.00),
(5, 'Eva Brown', 'IT', '2024-03-01', 2500.00),
(3, 'Carol Davis', 'Marketing', '2024-03-05', 1600.00),
(1, 'Alice Johnson', 'Sales', '2024-03-10', 2000.00),
(6, 'Frank Miller', 'IT', '2024-03-15', 2300.00),
(5, 'Eva Brown', 'IT', '2024-03-17', 2300.00);
"""

conn.execute(insert_data_query)
conn.commit()

# Display the data
run_sql("SELECT * FROM sales ORDER BY sale_date", "Sample Sales Data")


Sample Sales Data
 employee_id employee_name department  sale_date  sale_amount
           1 Alice Johnson      Sales 2024-01-15       1500.0
           2     Bob Smith      Sales 2024-01-16       2200.0
           3   Carol Davis  Marketing 2024-01-17       1800.0
           1 Alice Johnson      Sales 2024-02-10       1750.0
           4  David Wilson  Marketing 2024-02-12       2100.0
           2     Bob Smith      Sales 2024-02-15       1900.0
           5     Eva Brown         IT 2024-03-01       2500.0
           3   Carol Davis  Marketing 2024-03-05       1600.0
           1 Alice Johnson      Sales 2024-03-10       2000.0
           6  Frank Miller         IT 2024-03-15       2300.0
           5     Eva Brown         IT 2024-03-17       2300.0


,employee_id,employee_name,department,sale_date,sale_amount
0,1,Alice Johnson,Sales,2024-01-15,1500.0
1,2,Bob Smith,Sales,2024-01-16,2200.0
2,3,Carol Davis,Marketing,2024-01-17,1800.0
3,1,Alice Johnson,Sales,2024-02-10,1750.0
4,4,David Wilson,Marketing,2024-02-12,2100.0
5,2,Bob Smith,Sales,2024-02-15,1900.0
6,5,Eva Brown,IT,2024-03-01,2500.0
7,3,Carol Davis,Marketing,2024-03-05,1600.0
8,1,Alice Johnson,Sales,2024-03-10,2000.0
9,6,Frank Miller,IT,2024-03-15,2300.0


## Types of Window Functions

### 1. Ranking Functions

These functions assign ranks to rows within a partition.

In [3]:
# ROW_NUMBER(): Assigns unique sequential integers
query = """
SELECT 
    employee_name,
    department,
    sale_amount,
    ROW_NUMBER() OVER (ORDER BY sale_amount DESC) as row_num,
    ROW_NUMBER() OVER (PARTITION BY department ORDER BY sale_amount DESC) as dept_row_num
FROM sales
ORDER BY sale_amount DESC
"""

run_sql(query, "ROW_NUMBER() Example")


ROW_NUMBER() Example
employee_name department  sale_amount  row_num  dept_row_num
    Eva Brown         IT       2500.0        1             1
 Frank Miller         IT       2300.0        2             2
    Eva Brown         IT       2300.0        3             3
    Bob Smith      Sales       2200.0        4             1
 David Wilson  Marketing       2100.0        5             1
Alice Johnson      Sales       2000.0        6             2
    Bob Smith      Sales       1900.0        7             3
  Carol Davis  Marketing       1800.0        8             2
Alice Johnson      Sales       1750.0        9             4
  Carol Davis  Marketing       1600.0       10             3
Alice Johnson      Sales       1500.0       11             5


,employee_name,department,sale_amount,row_num,dept_row_num
0,Eva Brown,IT,2500.0,1,1
1,Frank Miller,IT,2300.0,2,2
2,Eva Brown,IT,2300.0,3,3
3,Bob Smith,Sales,2200.0,4,1
4,David Wilson,Marketing,2100.0,5,1
5,Alice Johnson,Sales,2000.0,6,2
6,Bob Smith,Sales,1900.0,7,3
7,Carol Davis,Marketing,1800.0,8,2
8,Alice Johnson,Sales,1750.0,9,4
9,Carol Davis,Marketing,1600.0,10,3


In [4]:
# RANK(): Assigns ranks with gaps for ties
query = """
SELECT 
    employee_name,
    department,
    sale_amount,
    RANK() OVER (ORDER BY sale_amount DESC) as rank_with_gaps,
    RANK() OVER (PARTITION BY department ORDER BY sale_amount DESC) as dept_rank
FROM sales
ORDER BY sale_amount DESC
"""

run_sql(query, "RANK() Example")


RANK() Example
employee_name department  sale_amount  rank_with_gaps  dept_rank
    Eva Brown         IT       2500.0               1          1
 Frank Miller         IT       2300.0               2          2
    Eva Brown         IT       2300.0               2          2
    Bob Smith      Sales       2200.0               4          1
 David Wilson  Marketing       2100.0               5          1
Alice Johnson      Sales       2000.0               6          2
    Bob Smith      Sales       1900.0               7          3
  Carol Davis  Marketing       1800.0               8          2
Alice Johnson      Sales       1750.0               9          4
  Carol Davis  Marketing       1600.0              10          3
Alice Johnson      Sales       1500.0              11          5


,employee_name,department,sale_amount,rank_with_gaps,dept_rank
0,Eva Brown,IT,2500.0,1,1
1,Frank Miller,IT,2300.0,2,2
2,Eva Brown,IT,2300.0,2,2
3,Bob Smith,Sales,2200.0,4,1
4,David Wilson,Marketing,2100.0,5,1
5,Alice Johnson,Sales,2000.0,6,2
6,Bob Smith,Sales,1900.0,7,3
7,Carol Davis,Marketing,1800.0,8,2
8,Alice Johnson,Sales,1750.0,9,4
9,Carol Davis,Marketing,1600.0,10,3


In [5]:
# DENSE_RANK(): Assigns ranks without gaps for ties
query = """
SELECT 
    employee_name,
    department,
    sale_amount,
    DENSE_RANK() OVER (ORDER BY sale_amount DESC) as dense_rank,
    DENSE_RANK() OVER (PARTITION BY department ORDER BY sale_amount DESC) as dept_dense_rank
FROM sales
ORDER BY sale_amount DESC
"""

run_sql(query, "DENSE_RANK() Example")


DENSE_RANK() Example
employee_name department  sale_amount  dense_rank  dept_dense_rank
    Eva Brown         IT       2500.0           1                1
 Frank Miller         IT       2300.0           2                2
    Eva Brown         IT       2300.0           2                2
    Bob Smith      Sales       2200.0           3                1
 David Wilson  Marketing       2100.0           4                1
Alice Johnson      Sales       2000.0           5                2
    Bob Smith      Sales       1900.0           6                3
  Carol Davis  Marketing       1800.0           7                2
Alice Johnson      Sales       1750.0           8                4
  Carol Davis  Marketing       1600.0           9                3
Alice Johnson      Sales       1500.0          10                5


,employee_name,department,sale_amount,dense_rank,dept_dense_rank
0,Eva Brown,IT,2500.0,1,1
1,Frank Miller,IT,2300.0,2,2
2,Eva Brown,IT,2300.0,2,2
3,Bob Smith,Sales,2200.0,3,1
4,David Wilson,Marketing,2100.0,4,1
5,Alice Johnson,Sales,2000.0,5,2
6,Bob Smith,Sales,1900.0,6,3
7,Carol Davis,Marketing,1800.0,7,2
8,Alice Johnson,Sales,1750.0,8,4
9,Carol Davis,Marketing,1600.0,9,3


In [6]:
# NTILE(): Divides rows into specified number of groups
query = """
SELECT 
    employee_name,
    sale_amount,
    NTILE(2) OVER (ORDER BY sale_amount DESC) as quartile,
    NTILE(4) OVER (ORDER BY sale_amount DESC) as quartile_4
FROM sales
ORDER BY sale_amount DESC
"""

run_sql(query, "NTILE() Example")


NTILE() Example
employee_name  sale_amount  quartile  quartile_4
    Eva Brown       2500.0         1           1
 Frank Miller       2300.0         1           1
    Eva Brown       2300.0         1           1
    Bob Smith       2200.0         1           2
 David Wilson       2100.0         1           2
Alice Johnson       2000.0         1           2
    Bob Smith       1900.0         2           3
  Carol Davis       1800.0         2           3
Alice Johnson       1750.0         2           3
  Carol Davis       1600.0         2           4
Alice Johnson       1500.0         2           4


,employee_name,sale_amount,quartile,quartile_4
0,Eva Brown,2500.0,1,1
1,Frank Miller,2300.0,1,1
2,Eva Brown,2300.0,1,1
3,Bob Smith,2200.0,1,2
4,David Wilson,2100.0,1,2
5,Alice Johnson,2000.0,1,2
6,Bob Smith,1900.0,2,3
7,Carol Davis,1800.0,2,3
8,Alice Johnson,1750.0,2,3
9,Carol Davis,1600.0,2,4


### 2. Aggregate Window Functions

These functions perform calculations across the window frame.

In [7]:
# SUM() with different window frames
query = """
SELECT 
    employee_name,
    sale_date,
    sale_amount, department,
    SUM(sale_amount) OVER () as total_sales,
    SUM(sale_amount) OVER (PARTITION BY department) as dept_total,
    SUM(sale_amount) OVER (ORDER BY sale_date ROWS UNBOUNDED PRECEDING) as running_total
FROM sales
ORDER BY sale_date
"""

run_sql(query, "SUM() Window Function Example")


SUM() Window Function Example
employee_name  sale_date  sale_amount department  total_sales  dept_total  running_total
Alice Johnson 2024-01-15       1500.0      Sales      21950.0      9350.0         1500.0
    Bob Smith 2024-01-16       2200.0      Sales      21950.0      9350.0         3700.0
  Carol Davis 2024-01-17       1800.0  Marketing      21950.0      5500.0         5500.0
Alice Johnson 2024-02-10       1750.0      Sales      21950.0      9350.0         7250.0
 David Wilson 2024-02-12       2100.0  Marketing      21950.0      5500.0         9350.0
    Bob Smith 2024-02-15       1900.0      Sales      21950.0      9350.0        11250.0
    Eva Brown 2024-03-01       2500.0         IT      21950.0      7100.0        13750.0
  Carol Davis 2024-03-05       1600.0  Marketing      21950.0      5500.0        15350.0
Alice Johnson 2024-03-10       2000.0      Sales      21950.0      9350.0        17350.0
 Frank Miller 2024-03-15       2300.0         IT      21950.0      7100.0      

,employee_name,sale_date,sale_amount,department,total_sales,dept_total,running_total
0,Alice Johnson,2024-01-15,1500.0,Sales,21950.0,9350.0,1500.0
1,Bob Smith,2024-01-16,2200.0,Sales,21950.0,9350.0,3700.0
2,Carol Davis,2024-01-17,1800.0,Marketing,21950.0,5500.0,5500.0
3,Alice Johnson,2024-02-10,1750.0,Sales,21950.0,9350.0,7250.0
4,David Wilson,2024-02-12,2100.0,Marketing,21950.0,5500.0,9350.0
5,Bob Smith,2024-02-15,1900.0,Sales,21950.0,9350.0,11250.0
6,Eva Brown,2024-03-01,2500.0,IT,21950.0,7100.0,13750.0
7,Carol Davis,2024-03-05,1600.0,Marketing,21950.0,5500.0,15350.0
8,Alice Johnson,2024-03-10,2000.0,Sales,21950.0,9350.0,17350.0
9,Frank Miller,2024-03-15,2300.0,IT,21950.0,7100.0,19650.0


In [8]:
# AVG() and COUNT() window functions
query = """
SELECT 
    employee_name,
    department,
    sale_amount,
    AVG(sale_amount) OVER () as overall_avg,
    AVG(sale_amount) OVER (PARTITION BY department) as dept_avg,
    COUNT(*) OVER (PARTITION BY department) as dept_count,
    sale_amount - AVG(sale_amount) OVER (PARTITION BY department) as diff_from_dept_avg
FROM sales
ORDER BY department, sale_amount DESC
"""

run_sql(query, "AVG() and COUNT() Window Functions Example")


AVG() and COUNT() Window Functions Example
employee_name department  sale_amount  overall_avg    dept_avg  dept_count  diff_from_dept_avg
    Eva Brown         IT       2500.0  1995.454545 2366.666667           3          133.333333
 Frank Miller         IT       2300.0  1995.454545 2366.666667           3          -66.666667
    Eva Brown         IT       2300.0  1995.454545 2366.666667           3          -66.666667
 David Wilson  Marketing       2100.0  1995.454545 1833.333333           3          266.666667
  Carol Davis  Marketing       1800.0  1995.454545 1833.333333           3          -33.333333
  Carol Davis  Marketing       1600.0  1995.454545 1833.333333           3         -233.333333
    Bob Smith      Sales       2200.0  1995.454545 1870.000000           5          330.000000
Alice Johnson      Sales       2000.0  1995.454545 1870.000000           5          130.000000
    Bob Smith      Sales       1900.0  1995.454545 1870.000000           5           30.000000
Alice 

,employee_name,department,sale_amount,overall_avg,dept_avg,dept_count,diff_from_dept_avg
0,Eva Brown,IT,2500.0,1995.454545,2366.666667,3,133.333333
1,Frank Miller,IT,2300.0,1995.454545,2366.666667,3,-66.666667
2,Eva Brown,IT,2300.0,1995.454545,2366.666667,3,-66.666667
3,David Wilson,Marketing,2100.0,1995.454545,1833.333333,3,266.666667
4,Carol Davis,Marketing,1800.0,1995.454545,1833.333333,3,-33.333333
5,Carol Davis,Marketing,1600.0,1995.454545,1833.333333,3,-233.333333
6,Bob Smith,Sales,2200.0,1995.454545,1870.000000,5,330.000000
7,Alice Johnson,Sales,2000.0,1995.454545,1870.000000,5,130.000000
8,Bob Smith,Sales,1900.0,1995.454545,1870.000000,5,30.000000
9,Alice Johnson,Sales,1750.0,1995.454545,1870.000000,5,-120.000000


In [9]:
# MIN() and MAX() window functions
query = """
SELECT 
    employee_name,
    department,
    sale_amount,
    MIN(sale_amount) OVER (PARTITION BY department) as dept_min,
    MAX(sale_amount) OVER (PARTITION BY department) as dept_max,
    MAX(sale_amount) OVER (ORDER BY sale_date ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) as local_max
FROM sales
ORDER BY department, sale_amount
"""

run_sql(query, "MIN() and MAX() Window Functions Example")


MIN() and MAX() Window Functions Example
employee_name department  sale_amount  dept_min  dept_max  local_max
 Frank Miller         IT       2300.0    2300.0    2500.0     2300.0
    Eva Brown         IT       2300.0    2300.0    2500.0     2300.0
    Eva Brown         IT       2500.0    2300.0    2500.0     2500.0
  Carol Davis  Marketing       1600.0    1600.0    2100.0     2500.0
  Carol Davis  Marketing       1800.0    1600.0    2100.0     2200.0
 David Wilson  Marketing       2100.0    1600.0    2100.0     2100.0
Alice Johnson      Sales       1500.0    1500.0    2200.0     2200.0
Alice Johnson      Sales       1750.0    1500.0    2200.0     2100.0
    Bob Smith      Sales       1900.0    1500.0    2200.0     2500.0
Alice Johnson      Sales       2000.0    1500.0    2200.0     2300.0
    Bob Smith      Sales       2200.0    1500.0    2200.0     2200.0


,employee_name,department,sale_amount,dept_min,dept_max,local_max
0,Frank Miller,IT,2300.0,2300.0,2500.0,2300.0
1,Eva Brown,IT,2300.0,2300.0,2500.0,2300.0
2,Eva Brown,IT,2500.0,2300.0,2500.0,2500.0
3,Carol Davis,Marketing,1600.0,1600.0,2100.0,2500.0
4,Carol Davis,Marketing,1800.0,1600.0,2100.0,2200.0
5,David Wilson,Marketing,2100.0,1600.0,2100.0,2100.0
6,Alice Johnson,Sales,1500.0,1500.0,2200.0,2200.0
7,Alice Johnson,Sales,1750.0,1500.0,2200.0,2100.0
8,Bob Smith,Sales,1900.0,1500.0,2200.0,2500.0
9,Alice Johnson,Sales,2000.0,1500.0,2200.0,2300.0


### 3. Value/Offset Functions

These functions access data from other rows relative to the current row.

In [10]:
# LAG() and LEAD() functions
query = """
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    LAG(sale_amount, 1) OVER (PARTITION BY employee_name ORDER BY sale_date) as prev_sale,
    LEAD(sale_amount, 1) OVER (PARTITION BY employee_name ORDER BY sale_date) as next_sale,
    sale_amount - LAG(sale_amount, 1) OVER (PARTITION BY employee_name ORDER BY sale_date) as sale_change
FROM sales
WHERE employee_name IN ('Alice Johnson', 'Bob Smith', 'Carol Davis')
ORDER BY employee_name, sale_date
"""

run_sql(query, "LAG() and LEAD() Functions Example")


LAG() and LEAD() Functions Example
employee_name  sale_date  sale_amount  prev_sale  next_sale  sale_change
Alice Johnson 2024-01-15       1500.0        NaN     1750.0          NaN
Alice Johnson 2024-02-10       1750.0     1500.0     2000.0        250.0
Alice Johnson 2024-03-10       2000.0     1750.0        NaN        250.0
    Bob Smith 2024-01-16       2200.0        NaN     1900.0          NaN
    Bob Smith 2024-02-15       1900.0     2200.0        NaN       -300.0
  Carol Davis 2024-01-17       1800.0        NaN     1600.0          NaN
  Carol Davis 2024-03-05       1600.0     1800.0        NaN       -200.0


,employee_name,sale_date,sale_amount,prev_sale,next_sale,sale_change
0,Alice Johnson,2024-01-15,1500.0,NaN,1750.0,NaN
1,Alice Johnson,2024-02-10,1750.0,1500.0,2000.0,250.0
2,Alice Johnson,2024-03-10,2000.0,1750.0,NaN,250.0
3,Bob Smith,2024-01-16,2200.0,NaN,1900.0,NaN
4,Bob Smith,2024-02-15,1900.0,2200.0,NaN,-300.0
5,Carol Davis,2024-01-17,1800.0,NaN,1600.0,NaN
6,Carol Davis,2024-03-05,1600.0,1800.0,NaN,-200.0


In [11]:
# FIRST_VALUE() and LAST_VALUE() functions
query = """
SELECT 
    employee_name,
    department,
    sale_date,
    sale_amount,
    FIRST_VALUE(sale_amount) OVER (PARTITION BY department ORDER BY sale_date) as first_dept_sale,
    LAST_VALUE(sale_amount) OVER (
        PARTITION BY department 
        ORDER BY sale_date 
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as last_dept_sale
FROM sales
ORDER BY department, sale_date
"""

run_sql(query, "FIRST_VALUE() and LAST_VALUE() Functions Example")


FIRST_VALUE() and LAST_VALUE() Functions Example
employee_name department  sale_date  sale_amount  first_dept_sale  last_dept_sale
    Eva Brown         IT 2024-03-01       2500.0           2500.0          2300.0
 Frank Miller         IT 2024-03-15       2300.0           2500.0          2300.0
    Eva Brown         IT 2024-03-17       2300.0           2500.0          2300.0
  Carol Davis  Marketing 2024-01-17       1800.0           1800.0          1600.0
 David Wilson  Marketing 2024-02-12       2100.0           1800.0          1600.0
  Carol Davis  Marketing 2024-03-05       1600.0           1800.0          1600.0
Alice Johnson      Sales 2024-01-15       1500.0           1500.0          2000.0
    Bob Smith      Sales 2024-01-16       2200.0           1500.0          2000.0
Alice Johnson      Sales 2024-02-10       1750.0           1500.0          2000.0
    Bob Smith      Sales 2024-02-15       1900.0           1500.0          2000.0
Alice Johnson      Sales 2024-03-10       2000.0

,employee_name,department,sale_date,sale_amount,first_dept_sale,last_dept_sale
0,Eva Brown,IT,2024-03-01,2500.0,2500.0,2300.0
1,Frank Miller,IT,2024-03-15,2300.0,2500.0,2300.0
2,Eva Brown,IT,2024-03-17,2300.0,2500.0,2300.0
3,Carol Davis,Marketing,2024-01-17,1800.0,1800.0,1600.0
4,David Wilson,Marketing,2024-02-12,2100.0,1800.0,1600.0
5,Carol Davis,Marketing,2024-03-05,1600.0,1800.0,1600.0
6,Alice Johnson,Sales,2024-01-15,1500.0,1500.0,2000.0
7,Bob Smith,Sales,2024-01-16,2200.0,1500.0,2000.0
8,Alice Johnson,Sales,2024-02-10,1750.0,1500.0,2000.0
9,Bob Smith,Sales,2024-02-15,1900.0,1500.0,2000.0


In [12]:
# NTH_VALUE() function
query = """
SELECT 
    employee_name,
    department,
    sale_amount,
    NTH_VALUE(sale_amount, 2) OVER (
        PARTITION BY department 
        ORDER BY sale_amount DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as second_highest_in_dept
FROM sales
ORDER BY department, sale_amount DESC
"""

run_sql(query, "NTH_VALUE() Function Example")


NTH_VALUE() Function Example
employee_name department  sale_amount  second_highest_in_dept
    Eva Brown         IT       2500.0                  2300.0
 Frank Miller         IT       2300.0                  2300.0
    Eva Brown         IT       2300.0                  2300.0
 David Wilson  Marketing       2100.0                  1800.0
  Carol Davis  Marketing       1800.0                  1800.0
  Carol Davis  Marketing       1600.0                  1800.0
    Bob Smith      Sales       2200.0                  2000.0
Alice Johnson      Sales       2000.0                  2000.0
    Bob Smith      Sales       1900.0                  2000.0
Alice Johnson      Sales       1750.0                  2000.0
Alice Johnson      Sales       1500.0                  2000.0


,employee_name,department,sale_amount,second_highest_in_dept
0,Eva Brown,IT,2500.0,2300.0
1,Frank Miller,IT,2300.0,2300.0
2,Eva Brown,IT,2300.0,2300.0
3,David Wilson,Marketing,2100.0,1800.0
4,Carol Davis,Marketing,1800.0,1800.0
5,Carol Davis,Marketing,1600.0,1800.0
6,Bob Smith,Sales,2200.0,2000.0
7,Alice Johnson,Sales,2000.0,2000.0
8,Bob Smith,Sales,1900.0,2000.0
9,Alice Johnson,Sales,1750.0,2000.0


## Window Frame Specifications

Window frames define which rows are included in the calculation for each row.

### Frame Types:
- `ROWS`: Physical number of rows
- `RANGE`: Logical range based on values

### Frame Boundaries:
- `UNBOUNDED PRECEDING`: From the start of the partition
- `UNBOUNDED FOLLOWING`: To the end of the partition
- `CURRENT ROW`: The current row
- `n PRECEDING`: n rows before the current row
- `n FOLLOWING`: n rows after the current row

In [13]:
# Different frame specifications
query = """
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    -- Running total from beginning
    SUM(sale_amount) OVER (
        ORDER BY sale_date 
        ROWS UNBOUNDED PRECEDING
    ) as running_total,
    -- Moving average of last 3 sales
    AVG(sale_amount) OVER (
        ORDER BY sale_date 
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) as moving_avg_3,
    -- Total of current and next 2 rows
    SUM(sale_amount) OVER (
        ORDER BY sale_date 
        ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING
    ) as forward_sum
FROM sales
ORDER BY sale_date
"""

run_sql(query, "Window Frame Specifications Example")


Window Frame Specifications Example
employee_name  sale_date  sale_amount  running_total  moving_avg_3  forward_sum
Alice Johnson 2024-01-15       1500.0         1500.0   1500.000000       5500.0
    Bob Smith 2024-01-16       2200.0         3700.0   1850.000000       5750.0
  Carol Davis 2024-01-17       1800.0         5500.0   1833.333333       5650.0
Alice Johnson 2024-02-10       1750.0         7250.0   1916.666667       5750.0
 David Wilson 2024-02-12       2100.0         9350.0   1883.333333       6500.0
    Bob Smith 2024-02-15       1900.0        11250.0   1916.666667       6000.0
    Eva Brown 2024-03-01       2500.0        13750.0   2166.666667       6100.0
  Carol Davis 2024-03-05       1600.0        15350.0   2000.000000       5900.0
Alice Johnson 2024-03-10       2000.0        17350.0   2033.333333       6600.0
 Frank Miller 2024-03-15       2300.0        19650.0   1966.666667       4600.0
    Eva Brown 2024-03-17       2300.0        21950.0   2200.000000       2300.0


,employee_name,sale_date,sale_amount,running_total,moving_avg_3,forward_sum
0,Alice Johnson,2024-01-15,1500.0,1500.0,1500.000000,5500.0
1,Bob Smith,2024-01-16,2200.0,3700.0,1850.000000,5750.0
2,Carol Davis,2024-01-17,1800.0,5500.0,1833.333333,5650.0
3,Alice Johnson,2024-02-10,1750.0,7250.0,1916.666667,5750.0
4,David Wilson,2024-02-12,2100.0,9350.0,1883.333333,6500.0
5,Bob Smith,2024-02-15,1900.0,11250.0,1916.666667,6000.0
6,Eva Brown,2024-03-01,2500.0,13750.0,2166.666667,6100.0
7,Carol Davis,2024-03-05,1600.0,15350.0,2000.000000,5900.0
8,Alice Johnson,2024-03-10,2000.0,17350.0,2033.333333,6600.0
9,Frank Miller,2024-03-15,2300.0,19650.0,1966.666667,4600.0


## Advanced Examples

### Finding Top N in Each Group

In [14]:
# Top 2 sales performers in each department
query = """
WITH ranked_sales AS (
    SELECT 
        employee_name,
        department,
        SUM(sale_amount) as total_sales,
        RANK() OVER (PARTITION BY department ORDER BY SUM(sale_amount) DESC) as dept_rank
    FROM sales
    GROUP BY employee_name, department
)
SELECT *
FROM ranked_sales
WHERE dept_rank <= 2
ORDER BY department, dept_rank
"""

run_sql(query, "Top 2 Sales Performers in Each Department")


Top 2 Sales Performers in Each Department
employee_name department  total_sales  dept_rank
    Eva Brown         IT       4800.0          1
 Frank Miller         IT       2300.0          2
  Carol Davis  Marketing       3400.0          1
 David Wilson  Marketing       2100.0          2
Alice Johnson      Sales       5250.0          1
    Bob Smith      Sales       4100.0          2


,employee_name,department,total_sales,dept_rank
0,Eva Brown,IT,4800.0,1
1,Frank Miller,IT,2300.0,2
2,Carol Davis,Marketing,3400.0,1
3,David Wilson,Marketing,2100.0,2
4,Alice Johnson,Sales,5250.0,1
5,Bob Smith,Sales,4100.0,2


In [15]:
# Percentage of total and cumulative percentage
query = """
SELECT 
    employee_name,
    department,
    sale_amount,
    ROUND(
        100.0 * sale_amount / SUM(sale_amount) OVER (), 
        2
    ) as pct_of_total,
    ROUND(
        100.0 * SUM(sale_amount) OVER (ORDER BY sale_amount DESC ROWS UNBOUNDED PRECEDING) / 
        SUM(sale_amount) OVER (), 
        2
    ) as cumulative_pct
FROM sales
ORDER BY sale_amount DESC
"""

run_sql(query, "Percentage of Total and Cumulative Percentage")


Percentage of Total and Cumulative Percentage
employee_name department  sale_amount  pct_of_total  cumulative_pct
    Eva Brown         IT       2500.0         11.39           11.39
 Frank Miller         IT       2300.0         10.48           21.87
    Eva Brown         IT       2300.0         10.48           32.35
    Bob Smith      Sales       2200.0         10.02           42.37
 David Wilson  Marketing       2100.0          9.57           51.94
Alice Johnson      Sales       2000.0          9.11           61.05
    Bob Smith      Sales       1900.0          8.66           69.70
  Carol Davis  Marketing       1800.0          8.20           77.90
Alice Johnson      Sales       1750.0          7.97           85.88
  Carol Davis  Marketing       1600.0          7.29           93.17
Alice Johnson      Sales       1500.0          6.83          100.00


,employee_name,department,sale_amount,pct_of_total,cumulative_pct
0,Eva Brown,IT,2500.0,11.39,11.39
1,Frank Miller,IT,2300.0,10.48,21.87
2,Eva Brown,IT,2300.0,10.48,32.35
3,Bob Smith,Sales,2200.0,10.02,42.37
4,David Wilson,Marketing,2100.0,9.57,51.94
5,Alice Johnson,Sales,2000.0,9.11,61.05
6,Bob Smith,Sales,1900.0,8.66,69.70
7,Carol Davis,Marketing,1800.0,8.20,77.90
8,Alice Johnson,Sales,1750.0,7.97,85.88
9,Carol Davis,Marketing,1600.0,7.29,93.17


## Complete List of Window Functions

### Ranking Functions:
- `ROW_NUMBER()`: Unique sequential integers
- `RANK()`: Ranking with gaps for ties
- `DENSE_RANK()`: Ranking without gaps for ties
- `NTILE(n)`: Divide rows into n groups

### Aggregate Functions (can be used as window functions):
- `SUM()`: Sum of values
- `AVG()`: Average of values
- `COUNT()`: Count of rows
- `MIN()`: Minimum value
- `MAX()`: Maximum value
- `STDDEV()`: Standard deviation
- `VARIANCE()`: Variance

### Value/Offset Functions:
- `LAG(expr, offset, default)`: Value from previous row
- `LEAD(expr, offset, default)`: Value from next row
- `FIRST_VALUE(expr)`: First value in window frame
- `LAST_VALUE(expr)`: Last value in window frame
- `NTH_VALUE(expr, n)`: nth value in window frame

### Distribution Functions:
- `PERCENT_RANK()`: Relative rank as percentage
- `CUME_DIST()`: Cumulative distribution
- `PERCENTILE_CONT(percentile)`: Continuous percentile
- `PERCENTILE_DISC(percentile)`: Discrete percentile

## Practice Quiz Questions

Try to solve these problems using window functions. Solutions are provided below each question.

### Question 1
Write a query to find each employee's sales amount and what percentage it represents of their department's total sales.

In [16]:
# Columns - employee_id employee_name department  sale_date  sale_amount

In [18]:
run_sql("SELECT * from sales", "all")


all
===
 employee_id employee_name department  sale_date  sale_amount
           1 Alice Johnson      Sales 2024-01-15       1500.0
           2     Bob Smith      Sales 2024-01-16       2200.0
           3   Carol Davis  Marketing 2024-01-17       1800.0
           1 Alice Johnson      Sales 2024-02-10       1750.0
           4  David Wilson  Marketing 2024-02-12       2100.0
           2     Bob Smith      Sales 2024-02-15       1900.0
           5     Eva Brown         IT 2024-03-01       2500.0
           3   Carol Davis  Marketing 2024-03-05       1600.0
           1 Alice Johnson      Sales 2024-03-10       2000.0
           6  Frank Miller         IT 2024-03-15       2300.0
           5     Eva Brown         IT 2024-03-17       2300.0


,employee_id,employee_name,department,sale_date,sale_amount
0,1,Alice Johnson,Sales,2024-01-15,1500.0
1,2,Bob Smith,Sales,2024-01-16,2200.0
2,3,Carol Davis,Marketing,2024-01-17,1800.0
3,1,Alice Johnson,Sales,2024-02-10,1750.0
4,4,David Wilson,Marketing,2024-02-12,2100.0
5,2,Bob Smith,Sales,2024-02-15,1900.0
6,5,Eva Brown,IT,2024-03-01,2500.0
7,3,Carol Davis,Marketing,2024-03-05,1600.0
8,1,Alice Johnson,Sales,2024-03-10,2000.0
9,6,Frank Miller,IT,2024-03-15,2300.0


In [19]:
# Try your solution here first!
# Uncomment and modify the query below:

query = """
select DISTINCT(employee_name), department, employee_sales, 100 * employee_sales / sum(sale_amount) OVER (PARTITION BY department) as percent from ( 
SELECT employee_name, department, sale_amount, SUM(sale_amount) OVER (PARTITION BY employee_name) as employee_sales from sales)"""
run_sql(query, "Question 1 - Your Solution")


Question 1 - Your Solution
employee_name department  employee_sales   percent
    Eva Brown         IT          4800.0 67.605634
 Frank Miller         IT          2300.0 32.394366
  Carol Davis  Marketing          3400.0 61.818182
 David Wilson  Marketing          2100.0 38.181818
Alice Johnson      Sales          5250.0 56.149733
    Bob Smith      Sales          4100.0 43.850267


,employee_name,department,employee_sales,percent
0,Eva Brown,IT,4800.0,67.605634
1,Frank Miller,IT,2300.0,32.394366
2,Carol Davis,Marketing,3400.0,61.818182
3,David Wilson,Marketing,2100.0,38.181818
4,Alice Johnson,Sales,5250.0,56.149733
5,Bob Smith,Sales,4100.0,43.850267


In [20]:
query = """SELECT 
    employee_name,
    department,
    sale_amount,
    ROUND(
        100.0 * sale_amount / SUM(sale_amount) OVER (PARTITION BY department), 
        2
    ) as pct_of_dept_sales
FROM sales
ORDER BY department, sale_amount DESC;"""
run_sql(query, "Question 1 - Their Solution")


Question 1 - Their Solution
employee_name department  sale_amount  pct_of_dept_sales
    Eva Brown         IT       2500.0              35.21
 Frank Miller         IT       2300.0              32.39
    Eva Brown         IT       2300.0              32.39
 David Wilson  Marketing       2100.0              38.18
  Carol Davis  Marketing       1800.0              32.73
  Carol Davis  Marketing       1600.0              29.09
    Bob Smith      Sales       2200.0              23.53
Alice Johnson      Sales       2000.0              21.39
    Bob Smith      Sales       1900.0              20.32
Alice Johnson      Sales       1750.0              18.72
Alice Johnson      Sales       1500.0              16.04


,employee_name,department,sale_amount,pct_of_dept_sales
0,Eva Brown,IT,2500.0,35.21
1,Frank Miller,IT,2300.0,32.39
2,Eva Brown,IT,2300.0,32.39
3,David Wilson,Marketing,2100.0,38.18
4,Carol Davis,Marketing,1800.0,32.73
5,Carol Davis,Marketing,1600.0,29.09
6,Bob Smith,Sales,2200.0,23.53
7,Alice Johnson,Sales,2000.0,21.39
8,Bob Smith,Sales,1900.0,20.32
9,Alice Johnson,Sales,1750.0,18.72


<details>
<summary>Click to see solution</summary>

```sql
SELECT 
    employee_name,
    department,
    sale_amount,
    ROUND(
        100.0 * sale_amount / SUM(sale_amount) OVER (PARTITION BY department), 
        2
    ) as pct_of_dept_sales
FROM sales
ORDER BY department, sale_amount DESC;
```
run_sql(solution_q1, "Question 1 Solution: Percentage of Department Sales")
</details>

### Question 2
For each sale, show the difference between the current sale amount and the previous sale amount for the same employee (ordered by date).

In [21]:
# Try your solution here first!
# Uncomment and modify the query below:
# LAG() and LEAD() functions


query = """
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    sale_amount - LAG(sale_amount, 1) over (PARTITION BY employee_name) as sale_diff
FROM sales
ORDER BY employee_name, sale_date
"""

run_sql(query, "Question 2 - Your Solution")


Question 2 - Your Solution
employee_name  sale_date  sale_amount  sale_diff
Alice Johnson 2024-01-15       1500.0        NaN
Alice Johnson 2024-02-10       1750.0      250.0
Alice Johnson 2024-03-10       2000.0      250.0
    Bob Smith 2024-01-16       2200.0        NaN
    Bob Smith 2024-02-15       1900.0     -300.0
  Carol Davis 2024-01-17       1800.0        NaN
  Carol Davis 2024-03-05       1600.0     -200.0
 David Wilson 2024-02-12       2100.0        NaN
    Eva Brown 2024-03-01       2500.0        NaN
    Eva Brown 2024-03-17       2300.0     -200.0
 Frank Miller 2024-03-15       2300.0        NaN


,employee_name,sale_date,sale_amount,sale_diff
0,Alice Johnson,2024-01-15,1500.0,NaN
1,Alice Johnson,2024-02-10,1750.0,250.0
2,Alice Johnson,2024-03-10,2000.0,250.0
3,Bob Smith,2024-01-16,2200.0,NaN
4,Bob Smith,2024-02-15,1900.0,-300.0
5,Carol Davis,2024-01-17,1800.0,NaN
6,Carol Davis,2024-03-05,1600.0,-200.0
7,David Wilson,2024-02-12,2100.0,NaN
8,Eva Brown,2024-03-01,2500.0,NaN
9,Eva Brown,2024-03-17,2300.0,-200.0


<details>
<summary>Click to see solution</summary>

```sql
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    LAG(sale_amount) OVER (PARTITION BY employee_name ORDER BY sale_date) as prev_sale,
    sale_amount - LAG(sale_amount) OVER (PARTITION BY employee_name ORDER BY sale_date) as sale_difference
FROM sales
ORDER BY employee_name, sale_date;
```
</details>

### Question 3
Rank employees by their total sales amount, and show only the top 3 performers overall.

<details>
<summary>💡 Click here for a hint</summary>
                      
```sql
query = """
WITH employee_totals AS (
    SELECT 
        employee_name,
        department,
        -- Add your aggregation and ranking here
    FROM sales
    GROUP BY employee_name, department
)
SELECT * FROM employee_totals
-- Add your WHERE clause here
"""
```
run_sql(query, "Question 3 - Your Solution")

</details>

In [39]:
# Try your solution here!

query = """
with ranked_sales AS (
    SELECT
        employee_name,
        SUM(sale_amount) as total_sales,
        RANK() OVER (ORDER BY SUM(sale_amount) DESC) as RANK
    FROM sales
    group BY employee_name, department
)
Select * from ranked_sales
where RANK <= 3
order by RANK;
"""

run_sql(query, "Question 3 - Your Solution")


Question 3 - Your Solution
employee_name  total_sales  RANK
Alice Johnson       5250.0     1
    Eva Brown       4800.0     2
    Bob Smith       4100.0     3


,employee_name,total_sales,RANK
0,Alice Johnson,5250.0,1
1,Eva Brown,4800.0,2
2,Bob Smith,4100.0,3


<details>
<summary>💡 Click here for a solution</summary>

```sql
query = """
WITH employee_totals AS (
    SELECT 
        employee_name,
        department,
        SUM(sale_amount) as total_sales,
        RANK() OVER (ORDER BY SUM(sale_amount) DESC) as sales_rank
    FROM sales
    GROUP BY employee_name, department
)
SELECT 
    sales_rank,
    employee_name,
    department,
    total_sales
FROM employee_totals
WHERE sales_rank <= 3
ORDER BY sales_rank
"""
```
run_sql(query, "Question 3 - Solution")
</details>

In [38]:
query = """
WITH employee_totals AS (
    SELECT 
        employee_name,
        department,
        SUM(sale_amount) as total_sales,
        RANK() OVER (ORDER BY SUM(sale_amount) DESC) as sales_rank
    FROM sales
    GROUP BY employee_name, department
)
SELECT 
    sales_rank,
    employee_name,
    department,
    total_sales
FROM employee_totals
WHERE sales_rank <= 3
ORDER BY sales_rank
"""
run_sql(query, "Question 3 - Solution")


Question 3 - Solution
 sales_rank employee_name department  total_sales
          1 Alice Johnson      Sales       5250.0
          2     Eva Brown         IT       4800.0
          3     Bob Smith      Sales       4100.0


,sales_rank,employee_name,department,total_sales
0,1,Alice Johnson,Sales,5250.0
1,2,Eva Brown,IT,4800.0
2,3,Bob Smith,Sales,4100.0


### Question 4
Calculate a running total of sales ordered by date, and show what percentage each running total represents of the final total.

<details>
<summary>💡 Click here for a hint</summary>

```sql
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    -- Add your running total calculation here
    -- Add your percentage calculation here
FROM sales
ORDER BY sale_date
```

Use `SUM() OVER()` with appropriate window frame and calculate percentage using the total.
</details>

In [51]:
# Try your solution here!

query = """
SELECT employee_name, department, sale_amount, sale_date, 
SUM(sale_amount) over (order by sale_date ASC ROWS UNBOUNDED PRECEDING) as running_total,
100 * SUM(sale_amount) over (order by sale_date ASC ROWS UNBOUNDED PRECEDING) / SUM(sale_amount) OVER () as running_pct from sales 
"""

run_sql(query, "Question 4 - Your Solution")


Question 4 - Your Solution
employee_name department  sale_amount  sale_date  running_total  running_pct
Alice Johnson      Sales       1500.0 2024-01-15         1500.0     6.833713
    Bob Smith      Sales       2200.0 2024-01-16         3700.0    16.856492
  Carol Davis  Marketing       1800.0 2024-01-17         5500.0    25.056948
Alice Johnson      Sales       1750.0 2024-02-10         7250.0    33.029613
 David Wilson  Marketing       2100.0 2024-02-12         9350.0    42.596811
    Bob Smith      Sales       1900.0 2024-02-15        11250.0    51.252847
    Eva Brown         IT       2500.0 2024-03-01        13750.0    62.642369
  Carol Davis  Marketing       1600.0 2024-03-05        15350.0    69.931663
Alice Johnson      Sales       2000.0 2024-03-10        17350.0    79.043280
 Frank Miller         IT       2300.0 2024-03-15        19650.0    89.521640
    Eva Brown         IT       2300.0 2024-03-17        21950.0   100.000000


,employee_name,department,sale_amount,sale_date,running_total,running_pct
0,Alice Johnson,Sales,1500.0,2024-01-15,1500.0,6.833713
1,Bob Smith,Sales,2200.0,2024-01-16,3700.0,16.856492
2,Carol Davis,Marketing,1800.0,2024-01-17,5500.0,25.056948
3,Alice Johnson,Sales,1750.0,2024-02-10,7250.0,33.029613
4,David Wilson,Marketing,2100.0,2024-02-12,9350.0,42.596811
5,Bob Smith,Sales,1900.0,2024-02-15,11250.0,51.252847
6,Eva Brown,IT,2500.0,2024-03-01,13750.0,62.642369
7,Carol Davis,Marketing,1600.0,2024-03-05,15350.0,69.931663
8,Alice Johnson,Sales,2000.0,2024-03-10,17350.0,79.043280
9,Frank Miller,IT,2300.0,2024-03-15,19650.0,89.521640


<details>
<summary>💡 Click here for solution </summary>

```sql
query = """
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    SUM(sale_amount) OVER (ORDER BY sale_date ROWS UNBOUNDED PRECEDING) as running_total,
    ROUND(
        SUM(sale_amount) OVER (ORDER BY sale_date ROWS UNBOUNDED PRECEDING) * 100.0 / 
        SUM(sale_amount) OVER (), 2
    ) as percentage_of_total
FROM sales
ORDER BY sale_date
"""

run_sql(query, "Question 4 - Solution")
```
<details>

In [49]:
query = """
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    SUM(sale_amount) OVER (ORDER BY sale_date ROWS UNBOUNDED PRECEDING) as running_total,
    ROUND(
        SUM(sale_amount) OVER (ORDER BY sale_date ROWS UNBOUNDED PRECEDING) * 100.0 / 
        SUM(sale_amount) OVER (), 2
    ) as percentage_of_total
FROM sales
ORDER BY sale_date
"""

run_sql(query, "Question 4 - Solution")


Question 4 - Solution
employee_name  sale_date  sale_amount  running_total  percentage_of_total
Alice Johnson 2024-01-15       1500.0         1500.0                 6.83
    Bob Smith 2024-01-16       2200.0         3700.0                16.86
  Carol Davis 2024-01-17       1800.0         5500.0                25.06
Alice Johnson 2024-02-10       1750.0         7250.0                33.03
 David Wilson 2024-02-12       2100.0         9350.0                42.60
    Bob Smith 2024-02-15       1900.0        11250.0                51.25
    Eva Brown 2024-03-01       2500.0        13750.0                62.64
  Carol Davis 2024-03-05       1600.0        15350.0                69.93
Alice Johnson 2024-03-10       2000.0        17350.0                79.04
 Frank Miller 2024-03-15       2300.0        19650.0                89.52
    Eva Brown 2024-03-17       2300.0        21950.0               100.00


,employee_name,sale_date,sale_amount,running_total,percentage_of_total
0,Alice Johnson,2024-01-15,1500.0,1500.0,6.83
1,Bob Smith,2024-01-16,2200.0,3700.0,16.86
2,Carol Davis,2024-01-17,1800.0,5500.0,25.06
3,Alice Johnson,2024-02-10,1750.0,7250.0,33.03
4,David Wilson,2024-02-12,2100.0,9350.0,42.60
5,Bob Smith,2024-02-15,1900.0,11250.0,51.25
6,Eva Brown,2024-03-01,2500.0,13750.0,62.64
7,Carol Davis,2024-03-05,1600.0,15350.0,69.93
8,Alice Johnson,2024-03-10,2000.0,17350.0,79.04
9,Frank Miller,2024-03-15,2300.0,19650.0,89.52


### Question 5
For each department, find the employee with the highest single sale and show how much higher it is than the department average.

<details>
<summary>💡 Click here for a hint</summary>

```sql
WITH dept_stats AS (
    SELECT 
        employee_name,
        department,
        sale_amount,
        -- Add your window functions here
    FROM sales
)
SELECT 
    department,
    employee_name,
    -- Add your calculations here
FROM dept_stats
-- Add your WHERE clause here
```

Use `RANK()` or `ROW_NUMBER()` to find the highest sale, and `AVG()` for department average.
</details>

In [81]:
# Try your solution here!

query = """
with sale_ranks as (
select employee_name, sale_amount, department, 
RANK() OVER (PARTITION by department order by sale_amount desc) as sale_rank, 
sale_amount - AVG(sale_amount) OVER (PARTITION BY department) as diff_from_avg
from sales)
select * from sale_ranks
where sale_rank = 1
"""

run_sql(query, "Question 5 - Your Solution")


Question 5 - Your Solution
employee_name  sale_amount department  sale_rank  diff_from_avg
    Eva Brown       2500.0         IT          1     133.333333
 David Wilson       2100.0  Marketing          1     266.666667
    Bob Smith       2200.0      Sales          1     330.000000


,employee_name,sale_amount,department,sale_rank,diff_from_avg
0,Eva Brown,2500.0,IT,1,133.333333
1,David Wilson,2100.0,Marketing,1,266.666667
2,Bob Smith,2200.0,Sales,1,330.000000


<details>
<summary>💡 Click here for a solution</summary>
    
```python
query = """
WITH dept_stats AS (
    SELECT 
        employee_name,
        department,
        sale_amount,
        RANK() OVER (PARTITION BY department ORDER BY sale_amount DESC) as sale_rank,
        AVG(sale_amount) OVER (PARTITION BY department) as dept_avg
    FROM sales
)
SELECT 
    department,
    employee_name,
    sale_amount as highest_sale,
    ROUND(dept_avg, 2) as dept_average,
    ROUND(sale_amount - dept_avg, 2) as above_average
FROM dept_stats
WHERE sale_rank = 1
ORDER BY department
"""
```
run_sql(query, "Question 5 - Solution")
</details>

In [75]:
query = """
WITH dept_stats AS (
    SELECT 
        employee_name,
        department,
        sale_amount,
        RANK() OVER (PARTITION BY department ORDER BY sale_amount DESC) as sale_rank,
        AVG(sale_amount) OVER (PARTITION BY department) as dept_avg
    FROM sales
)
SELECT 
    department,
    employee_name,
    sale_amount as highest_sale,
    ROUND(dept_avg, 2) as dept_average,
    ROUND(sale_amount - dept_avg, 2) as above_average
FROM dept_stats
WHERE sale_rank = 1
ORDER BY department
"""
run_sql(query, "Question 5 - Solution")


Question 5 - Solution
department employee_name  highest_sale  dept_average  above_average
        IT     Eva Brown        2500.0       2366.67         133.33
 Marketing  David Wilson        2100.0       1833.33         266.67
     Sales     Bob Smith        2200.0       1870.00         330.00


,department,employee_name,highest_sale,dept_average,above_average
0,IT,Eva Brown,2500.0,2366.67,133.33
1,Marketing,David Wilson,2100.0,1833.33,266.67
2,Sales,Bob Smith,2200.0,1870.00,330.00


### Question 6 (Advanced)
Create a query that shows each sale along with the 3-period moving average (current sale plus the 2 previous sales by date) for each employee.

<details>
<summary>💡 Click here for a hint</summary>

```sql
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    -- Add your moving average calculation here
FROM sales
ORDER BY employee_name, sale_date
```

Use `AVG() OVER()` with `PARTITION BY employee_name` and `ROWS 2 PRECEDING`.
</details>

In [111]:
# Try your solution here!

query = """
SELECT employee_name, 
sale_date,
sale_amount,
AVG(sale_amount) over (PARTITION BY employee_name ORDER BY sale_date ROWS BETWEEN 2 PRECEDING and CURRENT ROW) as a3_avg 
FROM sales
"""

run_sql(query, "Question 6 - Your Solution")


Question 6 - Your Solution
employee_name  sale_date  sale_amount  a3_avg
Alice Johnson 2024-01-15       1500.0  1500.0
Alice Johnson 2024-02-10       1750.0  1625.0
Alice Johnson 2024-03-10       2000.0  1750.0
    Bob Smith 2024-01-16       2200.0  2200.0
    Bob Smith 2024-02-15       1900.0  2050.0
  Carol Davis 2024-01-17       1800.0  1800.0
  Carol Davis 2024-03-05       1600.0  1700.0
 David Wilson 2024-02-12       2100.0  2100.0
    Eva Brown 2024-03-01       2500.0  2500.0
    Eva Brown 2024-03-17       2300.0  2400.0
 Frank Miller 2024-03-15       2300.0  2300.0


,employee_name,sale_date,sale_amount,a3_avg
0,Alice Johnson,2024-01-15,1500.0,1500.0
1,Alice Johnson,2024-02-10,1750.0,1625.0
2,Alice Johnson,2024-03-10,2000.0,1750.0
3,Bob Smith,2024-01-16,2200.0,2200.0
4,Bob Smith,2024-02-15,1900.0,2050.0
5,Carol Davis,2024-01-17,1800.0,1800.0
6,Carol Davis,2024-03-05,1600.0,1700.0
7,David Wilson,2024-02-12,2100.0,2100.0
8,Eva Brown,2024-03-01,2500.0,2500.0
9,Eva Brown,2024-03-17,2300.0,2400.0


<details>
<summary>💡 Click here for a solution</summary>


```SQL
query = """
SELECT 
    employee_name,
    sale_date,
    sale_amount,
    ROUND(
        AVG(sale_amount) OVER (
            PARTITION BY employee_name 
            ORDER BY sale_date 
            ROWS 2 PRECEDING
        ), 2
    ) as moving_avg_3_period
FROM sales
ORDER BY employee_name, sale_date
"""
```
run_sql(query, "Question 6 - Solution")
</details>

## Key Takeaways

1. **Window functions don't reduce the number of rows** - unlike GROUP BY aggregates
2. **PARTITION BY** divides the result set into groups for the window function
3. **ORDER BY** within OVER determines the order for ranking and frame calculations
4. **Frame specifications** (ROWS/RANGE) control which rows are included in calculations
5. **Ranking functions** handle ties differently (ROW_NUMBER vs RANK vs DENSE_RANK)
6. **LAG/LEAD** are perfect for comparing current row with previous/next rows
7. **Window functions can be combined** with CTEs for complex analytical queries

Window functions are essential for analytical SQL and are supported by most modern databases including PostgreSQL, SQL Server, Oracle, MySQL 8.0+, and SQLite 3.25+.

### Running This Notebook

To run this notebook:
1. Install dependencies: `pip install -r requirements.txt`
2. Start Jupyter: `jupyter notebook`
3. Run all cells in order

The notebook uses an in-memory SQLite database, so all data is temporary and will be lost when you close the notebook.

In [ ]:
# Clean up: Close the database connection
conn.close()
print("✅ Database connection closed. Notebook session complete!")